In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_kidney_disease"
cohort = "GSE180394"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_kidney_disease"
in_cohort_dir = "../../input/GEO/Chronic_kidney_disease/GSE180394"

# Output paths
out_data_file = "../../output/preprocess/Chronic_kidney_disease/GSE180394.csv"
out_gene_data_file = "../../output/preprocess/Chronic_kidney_disease/gene_data/GSE180394.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE180394.csv"
json_path = "../../output/preprocess/Chronic_kidney_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import numpy as np
import os
import re
from typing import Dict, List, Optional, Callable, Any, Tuple

# 1. Gene Expression Data Availability
# From the background information, this is gene expression data from Affymetrix microarray
# "Profiling was performed on Affymetrix ST2.1 microarray platform"
is_gene_available = True

# 2.1 Data Availability
# For trait:
# The sample group includes different kidney diseases and living donors
trait_row = 0  # This corresponds to 'sample group' in the sample characteristics

# For age:
# No age information is available in the characteristics
age_row = None

# For gender:
# No gender information is available in the characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value_str):
    if not isinstance(value_str, str):
        return None
    
    # Extract the value after colon
    if ":" in value_str:
        value = value_str.split(":", 1)[1].strip()
    else:
        value = value_str.strip()
    
    # Binary classification: Living donor (0) vs CKD (1)
    if "Living donor" in value:
        return 0  # Control
    elif any(term in value for term in ["DN", "FSGS", "GN", "IgAN", "Nephritis", "Hypertensive Nephrosclerosis", 
                                        "Light-Chain Deposit Disease", "LN-WHO", "MCD", "MN", "CKD", 
                                        "Interstitial fibrosis", "Thin-BMD"]):
        return 1  # CKD patient
    elif "Tumor Nephrectomy" in value:
        # These are unaffected parts from tumor nephrectomy, likely normal kidney tissue
        return 0
    
    return None  # Unknown or undefined

# The following functions are defined as placeholders since the data is not available
def convert_age(value_str):
    return None

def convert_gender(value_str):
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial validation
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We only process this step if clinical data is available
if trait_row is not None:
    # Convert the sample characteristics dictionary to a DataFrame
    # The dictionary is in the format {row_index: [values_for_samples]}
    # We need to create a DataFrame where each row is a feature and each column is a sample
    
    # Sample characteristics from the previous output
    sample_char_dict = {
        0: ['sample group: Living donor', "sample group: 2' FSGS", 'sample group: chronic Glomerulonephritis (GN) with infiltration by CLL', 
            'sample group: DN', 'sample group: FGGS', 'sample group: FSGS', 'sample group: Hydronephrosis', 'sample group: IgAN', 
            'sample group: Interstitial nephritis', 'sample group: Hypertensive Nephrosclerosis', 
            'sample group: Light-Chain Deposit Disease (IgG lambda)', 'sample group: LN-WHO III', 'sample group: LN-WHO III+V', 
            'sample group: LN-WHO IV', 'sample group: LN-WHO IV+V', 'sample group: LN-WHO V', 'sample group: LN-WHO-I/II', 
            'sample group: MCD', 'sample group: MN', 'sample group: CKD with mod-severe Interstitial fibrosis', 
            'sample group: Thin-BMD', 'sample group: Unaffected parts of Tumor Nephrectomy'],
        1: ['tissue: Tubuli from kidney biopsy']
    }
    
    # Create a DataFrame from the dictionary
    # Each key in the dictionary becomes a row in the DataFrame
    clinical_data = pd.DataFrame.from_dict(sample_char_dict, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    for key, value in preview.items():
        print(f"{key}: {value}")
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Assume gene data is available

# Extract gene data
try:
    # Extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # Print the first 20 gene/probe identifiers
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20].tolist())
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print(f"File path: {matrix_file}")
    print("Please check if the file exists and contains the expected markers.")


### Step 4: Gene Identifier Review

In [ ]:
# Reviewing the gene identifiers
# The identifiers follow the pattern "number_at" which is characteristic of Affymetrix probe IDs
# These are not standard human gene symbols and need to be mapped
# For example, '100009613_at' is an Affymetrix probe ID, not a standard gene symbol like "BRCA1"

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3].to_string())

# Check if there are any columns that might contain gene information beyond what we've seen
potential_gene_columns = [col for col in gene_annotation.columns if 
                          any(term in col.upper() for term in ["GENE", "SYMBOL", "NAME", "ID"])]
print(f"\nPotential gene-related columns: {potential_gene_columns}")

# Look for additional columns that might contain gene symbols
# Since we only have 'ID' and 'ENTREZ_GENE_ID', check if we need to use Entrez IDs for mapping
gene_id_col = 'ID'
gene_symbol_col = None

# Check various potential column names for gene symbols
for col_name in ['GENE_SYMBOL', 'SYMBOL', 'GENE', 'GENE_NAME', 'GB_ACC']:
    if col_name in gene_annotation.columns:
        gene_symbol_col = col_name
        break

# If no dedicated symbol column is found, we'll need to use ENTREZ_GENE_ID
if gene_symbol_col is None and 'ENTREZ_GENE_ID' in gene_annotation.columns:
    gene_symbol_col = 'ENTREZ_GENE_ID'
    print("\nNo direct gene symbol column found. Will use Entrez Gene IDs for mapping.")

if gene_id_col in gene_annotation.columns and gene_symbol_col is not None:
    print(f"\nSample mappings from '{gene_id_col}' to '{gene_symbol_col}':")
    sample_mappings = gene_annotation[[gene_id_col, gene_symbol_col]].head(10)
    print(sample_mappings)
    
    # Check for non-null mappings to confirm data quality
    non_null_mappings = gene_annotation[[gene_id_col, gene_symbol_col]].dropna(subset=[gene_symbol_col])
    print(f"\nNumber of probes with gene ID mappings: {len(non_null_mappings)}")
    print(f"Sample of valid mappings:")
    print(non_null_mappings.head(5))
else:
    print("Required mapping columns not found in the annotation data. Will need to explore alternative mapping approaches.")


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine which column in gene annotation corresponds to gene identifiers and which to gene symbols
# From previous analysis, the gene annotation has 'ID' for probe IDs and 'ENTREZ_GENE_ID' for Entrez Gene IDs
probe_col = 'ID'
gene_col = 'ENTREZ_GENE_ID'

# 2. Extract the two columns from the gene annotation dataframe to create the mapping dataframe
print("Creating gene mapping DataFrame...")
mapping_data = get_gene_mapping(gene_annotation, probe_col, gene_col)
print(f"Created mapping between {probe_col} and {gene_col}")
print(f"Mapping data shape: {mapping_data.shape}")
print("Sample of mapping data:")
print(mapping_data.head())

# 3. We need to ensure our mapping works by examining the data formats
# Let's get a fresh copy of the gene expression data
gene_data = get_genetic_data(matrix_file)
print(f"Gene expression data shape: {gene_data.shape}")

# Create a custom mapping approach
# First, get the overlap of probe IDs between expression data and annotation
common_probes = set(gene_data.index) & set(mapping_data['ID'])
print(f"Number of probes in expression data: {len(gene_data.index)}")
print(f"Number of probes in mapping data: {len(mapping_data['ID'])}")
print(f"Number of common probes: {len(common_probes)}")

# Filter mapping to only include probes that exist in expression data
valid_mapping = mapping_data[mapping_data['ID'].isin(common_probes)]
print(f"Valid mapping shape after filtering: {valid_mapping.shape}")

# Create a direct mapping from probe ID to Entrez Gene ID
probe_to_gene = {}
for idx, row in valid_mapping.iterrows():
    probe_id = row['ID'] 
    gene_id = str(row['Gene'])  # Convert to string
    
    if probe_id not in probe_to_gene:
        probe_to_gene[probe_id] = []
    probe_to_gene[probe_id].append(gene_id)

# Create a new gene expression DataFrame
result = pd.DataFrame()

# For each probe, distribute its expression to its mapped genes
for probe_id, gene_ids in probe_to_gene.items():
    if not gene_ids:  # Skip if no genes mapped
        continue
        
    # Get probe expression data
    probe_expr = gene_data.loc[probe_id]
    
    # Distribute expression equally among genes
    weight = 1.0 / len(gene_ids)
    
    for gene_id in gene_ids:
        # Skip empty gene IDs
        if not gene_id or gene_id == 'nan':
            continue
            
        # Add weighted expression to the gene
        if gene_id in result.index:
            result.loc[gene_id] += probe_expr * weight
        else:
            result.loc[gene_id] = probe_expr * weight

print(f"Converted probe-level data to gene-level expression")
print(f"Gene data shape after mapping: {result.shape}")

if not result.empty:
    print("First 10 gene symbols after mapping:")
    print(result.index[:10].tolist())
    
    # Check for top genes with highest expression to verify data quality
    print("\nGenes with highest mean expression:")
    mean_expression = result.mean(axis=1).sort_values(ascending=False)
    print(mean_expression.head(10))
    
    # Update gene_data with our processed result
    gene_data = result
else:
    print("\nWarning: No genes were mapped. Check the mapping process.")


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Determine which column in gene annotation corresponds to gene identifiers and which to gene symbols
# From previous analysis, the gene annotation has 'ID' for probe IDs and 'ENTREZ_GENE_ID' for Entrez Gene IDs
prob_col = 'ID'
gene_col = 'ENTREZ_GENE_ID'

# 2. Extract the two columns from the gene annotation dataframe to create the mapping dataframe
print("Creating gene mapping DataFrame...")
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Created mapping between {prob_col} and {gene_col}")
print(f"Mapping data shape: {mapping_data.shape}")
print("Sample of mapping data:")
print(mapping_data.head())

# 3. Convert probe-level measurements to gene expression data by applying the gene mapping
print("Converting probe-level measurements to gene-level expression data...")

# Use the library function to convert probe-level data to gene-level expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Resulting gene expression data shape: {gene_data.shape}")

# Show a sample of the resulting gene data
print("Sample of gene expression data:")
if not gene_data.empty:
    print("First 10 gene symbols after mapping:")
    print(gene_data.index[:10].tolist())
    
    # Check for top genes with highest expression to verify data quality
    print("\nGenes with highest mean expression:")
    mean_expression = gene_data.mean(axis=1).sort_values(ascending=False)
    print(mean_expression.head(10))
else:
    print("WARNING: No genes were mapped successfully.")